In [1]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import numpy as np
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence
import pandas as pd
from multiprocessing import Pool
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
class EpochSaver(CallbackAny2Vec):
	def __init__(self,path_prefix):
		self.epoch=0
	def on_epoch_end(self,model):
		print("Epoch_"+str(self.epoch))
		print("Training loss:"+str(model.get_latest_training_loss()))
		self.epoch+=1
epoch_saver=EpochSaver("model_some")

In [ ]:
# model = Word2Vec(min_count=2)
# model.build_vocab(sentences=sentences)
# vocabularies = list(model.wv.vocab.keys())
# EN_vocab = [x for x in vocabularies if '_EN_' in x]
# len(EN_vocab)

In [ ]:
model.wv.most_similar(['shit_EN_noun'], topn=10)

[('merde_FR_noun', 0.9669520854949951),
 ('mierda_ES_noun', 0.9584709405899048),
 ('merda_SC_noun', 0.9510030150413513),
 ('mierda_AN_noun', 0.9509536623954773),
 ('căcat_RO_noun', 0.9508905410766602),
 ('rahat_RO_noun', 0.9503856897354126),
 ('merda_IT_noun', 0.9464212656021118),
 ('merdo_EO_noun', 0.9322292804718018),
 ('kaka_EU_noun', 0.9319580793380737),
 ('mafa_GL_noun', 0.9318681955337524)]

In [ ]:
model = Word2Vec.load('/content/drive/MyDrive/word2vec_imple/model_word2vec/model_word2vec_skipgram_100_shord_dict')

In [ ]:
model.wv.most_similar(['Artini_FR_properNoun'], topn=10)

[('Artini_IT_properNoun', 0.9939879775047302),
 ('Artini_PT_properNoun', 0.9932299256324768),
 ('kunakuzito_EO_noun', 0.8983769416809082),
 ('Vance_EN_properNoun', 0.8957081437110901),
 ('Susinos_del_Páramo_GL_properNoun', 0.8948129415512085),
 ('nivo-pluvial_FR_adjective', 0.893297553062439),
 ('sexològic_CA_adjective', 0.8929303288459778),
 ('Jaxon_CA_properNoun', 0.8922513723373413),
 ('Cousy_FR_properNoun', 0.8897876143455505),
 ('curculiónido_ES_noun', 0.8889787197113037)]

In [ ]:
vocabularies = list(model.wv.vocab.keys())

In [ ]:
EN_vocab = [x for x in vocabularies if '_EN_' in x]
EN_vectors = model.wv[EN_vocab]

FR_vocab = [x for x in vocabularies if '_FR_' in x]
FR_vectors = model.wv[FR_vocab]

PT_vocab = [x for x in vocabularies if '_PT_' in x]
PT_vectors = model.wv[PT_vocab]

In [ ]:
PT_vocab[0:10]

['proprietário_PT_noun',
 'abono_PT_noun',
 'cercania_PT_noun',
 'ancorar_PT_verb',
 'quarenta_PT_numeral',
 'monitorização_PT_noun',
 'Lus_PT_properNoun',
 'regular_PT_adjective',
 'refúgio_PT_noun',
 'visibilidade_PT_noun']

In [ ]:
print('FR vocal len: ' +str(len(PT_vocab)))
print('PT vocal len: ' +str(len(FR_vocab)))
print('EN vocal len: ' +str(len(EN_vocab)))

FR vocal len: 51033
PT vocal len: 82200
EN vocal len: 59577


In [ ]:
a = 'rowing_club_of_Hondarribia_EN_properNoun'
subs = a.split('_')
pos = subs[-1]
" ".join(subs[0:len(subs)-2])

'rowing club of Hondarribia'

In [ ]:
def get_word_pos(x):
  subs = x.split('_')
  word = " ".join(subs[0:len(subs)-2])
  pos = subs[-1]
  return word, pos

In [ ]:
target_vocab = FR_vocab
target_vectors = FR_vectors
def find_translate(in_word):
  source_vec = model.wv[in_word]
  word_source, pos_source = get_word_pos(in_word)
  cosine_cosine_simi = model.wv.cosine_similarities(source_vec, target_vectors)
  sorted_index = np.argsort(-cosine_cosine_simi)
  for index in sorted_index:
    word_target, pos_target = get_word_pos(target_vocab[index])
    if pos_source == pos_target:
      return word_source + '\t' + word_target + '\t' + pos_source + '\t' + str(np.round(cosine_cosine_simi[index],2))

In [ ]:
# array = []
# for en_word in EN_vocab:
#   array.append(find_translate(en_word, FR_vocal, FR_vectors))

In [ ]:
def to_word_test(dataframe, lang):
  dataframe['source'] = [str(x) for x in dataframe['source']] 
  words = dataframe['source'] +'_' + lang + '_' + dataframe['POS']
  return list(words.values) 

In [ ]:
all_pair = ['', 'en_pt', 'fr_en', 'fr_pt', 'pt_en', 'pt_fr']
en_fr = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_en_fr.csv'), 'EN') 
en_pt = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_en_pt.csv'), 'EN') 
fr_en = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_fr_en.csv'), 'FR') 
fr_pt = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_fr_pt.csv'), 'FR') 
pt_en = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_pt_en.csv'), 'PT') 
pt_fr = to_word_test(pd.read_csv('/content/drive/MyDrive/data/Dictionary_To_Infe/to_infe_pt_fr.csv'), 'PT') 

'EN'

In [ ]:
len(list(set(EN_vocab)&set(en_fr)))

In [ ]:
  
target_vocab = FR_vocab
target_vectors = FR_vectors

p = Pool(5)
array = p.map(find_translate, EN_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_en_fr.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)

p = Pool(5)
array = p.map(find_translate, PT_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_pt_fr.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)


target_vocab = EN_vocab
target_vectors = EN_vectors

p = Pool(5)
array = p.map(find_translate, FR_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_fr_en.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)

p = Pool(5)
array = p.map(find_translate, PT_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_pt_en.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)

target_vocab = PT_vocab
target_vectors = PT_vectors

p = Pool(5)
array = p.map(find_translate, EN_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_en_pt.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)


p = Pool(5)
array = p.map(find_translate, FR_vocab)  
with open('/content/drive/MyDrive/word2vec_imple/build_dict_infr_skip_gram/dict_fr_pt.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)

In [ ]:
#build pt->fr
from multiprocessing import Pool
p = Pool(5)
array = p.map(find_translate, PT_vocab)  
with open('/content/drive/MyDrive/model/dict_pt_fr.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)


KeyboardInterrupt

  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-35:
Process ForkPoolWorker-34:
Process ForkPoolWorker-33:
Process ForkPoolWorker-32:
Process ForkPoolWorker-31:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
